# Submission Notebook

In [1]:
import pandas as pd

In [2]:
orig = pd.read_csv("../data/kaggle_survey_2020_responses.csv", low_memory=False)

# 1) First part: tidy data

In [3]:
# the first row contains the question. # This is useful to read but it is not actual data:
df = orig.loc[1:]
# df.head(2)

In [11]:
# Create background dataframes. Faster, efficient, better looking
# For and dataset that you plan to work on more than a day: Chose appropriate column names
demographics = df[["Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q15"]]
demographics.columns = ["age", "gender", "country", "education", "role", "code experience"]
demographics.head(2)

,age,gender,country,education,role,experience
1,35-39,Man,Colombia,Doctoral degree,Student,5-10 years
2,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years


### According to the Manual, the Survey essentially contains four parts.

First part is six general primary demographic (sample identification) questions (Q1 to Q6). In questionnaire surveys, such question help identify the characteristics on the population that may explain other features. Primary demographic variables amy provide info about secondary exlonatory variables (could be examined from a dependent and explonatory point of view) and variables that we might want to treat as depedent variables.

The second part contains two subparts:
a) 5 general questions about software tools:

    Q7 and Q8: programming language, Q9: IDE,
    Q10:hosted notebook products, Important note: Not all free and we should expect to gain different info from the different categories.
    and Q14: Data Visualization Libraries

and 3 general questions about hardware Q11: Computing platform, Q12: types of specalized hardware, Q13 More Specialized Hardware (TPU). 
To the extend that such question may overlap, they may be used to assess the quality of the answer, whether this is intented by desing or not.

The third part focuses on ML.
    
    Q15 is ML experience which may be grouped together with other general demopraphic variables.
    Q16: is about ML libraries which may be also grouped together with programming software questions.
    Q17: is about ML Algos in general
    Q18: is about Computer vision (subset of Q17)
    Q19: is about NLP methods (subset of Q17)

The fourth part is about Company of employment and work demographics:
    
    Q20: Company size
    Q21: Company DS time
    Q22: Company and ML methods employment
    Q23: Job role

The 5th part is compensation and cost:
    
    Q24: Your compensation = employers cost.
    Q25: ML and cloud computing services cost.
    Q26-A:
    

    



In [5]:
# Create functions for reuseable code:
# Function to join multiple choice columns
def filter_df(df, question_index):
    columns = [col for col in df.columns if col.startswith(f"Q{question_index}_")]
    df = df.loc[1:][columns].fillna(False)
    unique_values = []
    for column in columns:
        column_unique = list(df[column].unique())
        column_unique.remove(False)
        unique_values.append(column_unique[0])
    filtered_df = df.replace(to_replace=unique_values, value=True)
    filtered_df.columns = [value.strip() for value in unique_values]
    return filtered_df

In [6]:
prog = filter_df(df, 7)
# prog

In [7]:
# We will usually want to join one or more of the filtered dataframes to the background one:
def join_dfs(left: pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
    df = pd.concat([left, right], axis=1, ignore_index=False)
    return df

In [8]:
prog = join_dfs(demographics, prog)
prog.head(3)

,age,gender,country,education,title,experience,Python,R,SQL,C,C++,Java,Javascript,Julia,Swift,Bash,MATLAB,None,Other
1,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,True,True,True,True,False,False,True,False,False,False,True,False,True
2,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,True,True,True,False,False,False,False,False,False,False,False,False,False
3,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,False,False,False,False,False,True,True,False,False,True,False,False,False


# Comments Section (draft, unordered, raw)

First remarks on Age:
Age groups could perhaps be left for competitors to construct.
Data could be presented as answered. It is interesting to see the large number of the 20-21 class, but some useful information may be lost.

Even more problematic:
There are 7 bins (25-29 until 55-59) that contain 5year-periods, one bin 60-69 that includes 10 years and 70+. While it is interesting that there are 76 responses 70 or more year olds, we do not have the max, which I find disappointing because you know, we all need some inspiration!
The real interesting part though is why the bin 18-21 includes 4 years and bin 22-24 includes only 3 ?

What would be the age distribution be if we had an equal length bin 20-24 and a bin under 20 ?
Let us explore this idea:
First of all, it is easy to show that the largest group would be 20 to 24 by a wide margin.
And this is a very interesting result on which all other info has to be conditioned upon.
First I will show that under all assumptions this assumption about the 20-25 age stands.

If 18-21 are evenly  distributed: => more or less 1156 per year => 2300 20 and 21 => age group: 20-24 = 3786 + 2300 = 6000 !

It seems that Q4 is another case where the data are a root of possible errors.
The question is stated in way a that groups together those who have a degree and those who plan to attain one.

Therefore, it would be a major mistake to reach to robust conclusions about the relation about the educational level and other variable such as salary or position.

A mere intention to attain a degree may convey useful information about job position or salary, but that  certainly would not have the same effects as having one.

Arguably, Q4 could be split to two distinct questions, since it contains two distinct question.

In any case, Q4 poses the second riddle that we need to explore. How can we tell who has a degree and who hasn't by exploring the data?

Once, we have that then we could make meaningful explorations about the educational level and its relation to other variables. 